In [1]:
import os
import csv
import subprocess as sub
import shutil
#Me ubico en el directorio que quiero 
os.chdir('/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/')


In [4]:
for i in range(500):
    #j = i+1
    with open("cerrada-rep2-frame"+str(i+1)+".pdb", 'r') as file:
        filedata = file.read()

    # Replace the target string  
    #k = j+1
    filedata = filedata.replace("MODEL      "+str(i+2), 'MODEL        1')

    # Write the file out again
    with open("cerrada-rep2-frame"+str(i+1)+".pdb", 'w') as file:
        file.write(filedata)
        

In [5]:
src_path = r"/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/report.xml"
dst_path = r"/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/reports/report.xml"
for i in range(501):
    os.chdir('/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/')
    sub.call(["plip", "-f", "cerrada-rep2-frame"+str(i)+".pdb", "-yv", "-x"]) 
    shutil.move(src_path, dst_path)
    os.rename("reports/report.xml", "reports/frame"+str(i)+"_report.xml")
    pse_src_path = r"/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/CERRADA_REP2_FRAME"+str(i)+"_PROTEIN_1SY_A_540.pse"
    pse_dst_path = r"/home/hoseki/Desktop/Sysbio/cSTING/cerrada-rep2/interactions/pse/CERRADA_REP2_FRAME"+str(i)+"_PROTEIN_1SY_A_540.pse"
    shutil.move(pse_src_path, pse_dst_path)
    
print("Finished bitches")

Finished bitches


In [6]:
### 1 - LIBRARIES ###
import os
import csv
import subprocess as sub
import pandas as pd 
from lxml import objectify
import xmltodict as xm
import re
import numpy as np

In [7]:
hydrogen_bonds = pd.DataFrame(columns = ["id", "resid", "restype", "donor_group", "acceptor_group", "count", "Occupancy"])
salt_bridges = pd.DataFrame(columns = ["id", "resid", "restype", "count", "occupancy"])

for i in range(501):
    data = open('reports/frame'+str(i)+'_report.xml', 'r').read() #abro el archivo xml
    data2 = xm.parse(data) #leo cuasi bien el fucking archivo
    hb_aux = pd.DataFrame.from_dict(data2['report']['bindingsite']['interactions']['hydrogen_bonds']["hydrogen_bond"])
    sb_aux = pd.DataFrame.from_dict(data2['report']['bindingsite']['interactions']['salt_bridges']["salt_bridge"])
    if i == 0:
        for ind in hb_aux.index:
            hb_new_row = {"id":str(hb_aux["donoridx"][ind]+hb_aux["acceptoridx"][ind]), 
                          "restype":str(hb_aux["restype"][ind]), 
                          "resid":str(hb_aux["resnr"][ind]), 
                          "donor_group":str(hb_aux["donortype"][ind]), 
                          "acceptor_group":str(hb_aux["acceptortype"][ind]), 
                          "count":1, 
                          "Occupancy":0}
            hydrogen_bonds = hydrogen_bonds.append(hb_new_row, ignore_index=True)
        for ind in sb_aux.index:
            sb_new_row = {"id":str(sb_aux["resnr"][ind]+sb_aux["resnr_lig"][ind]), 
                          "resid":str(sb_aux["resnr"][ind]), 
                          "restype":str(sb_aux["restype"][ind]), 
                          "count":1, 
                          "occupancy":0}
            salt_bridges = salt_bridges.append(sb_new_row, ignore_index=True)
            
    else:
        for ind in hb_aux.index:
            hb_id = str(hb_aux["donoridx"][ind]+hb_aux["acceptoridx"][ind])
            if hb_id in set(hydrogen_bonds.id):
                hydrogen_bonds['count'][hydrogen_bonds.index[hydrogen_bonds['id'] == hb_id].tolist()] = hydrogen_bonds['count'][hydrogen_bonds.index[hydrogen_bonds['id'] == hb_id].tolist()] + 1
            else:
                hb_new_row = {"id":str(hb_aux["donoridx"][ind]+hb_aux["acceptoridx"][ind]), 
                              "restype":str(hb_aux["restype"][ind]), 
                              "resid":str(hb_aux["resnr"][ind]), 
                              "donor_group":str(hb_aux["donortype"][ind]), 
                              "acceptor_group":str(hb_aux["acceptortype"][ind]), 
                              "count":1, 
                              "Occupancy":0}
                hydrogen_bonds = hydrogen_bonds.append(hb_new_row, ignore_index=True)
        for ind in sb_aux.index:
            sb_id = str(sb_aux["resnr"][ind]+sb_aux["resnr_lig"][ind])
            if sb_id in set(salt_bridges.id):
                salt_bridges['count'][salt_bridges.index[salt_bridges['id'] == sb_id].tolist()] = salt_bridges['count'][salt_bridges.index[salt_bridges['id'] == sb_id].tolist()] + 1
            else:
                sb_new_row = {"id":str(sb_aux["resnr"][ind]+sb_aux["resnr_lig"][ind]), 
                              "resid":str(sb_aux["resnr"][ind]), 
                              "restype":str(sb_aux["restype"][ind]), 
                              "count":1, 
                              "occupancy":0}
                salt_bridges = salt_bridges.append(sb_new_row, ignore_index=True)        

hydrogen_bonds['Occupancy'] = hydrogen_bonds['count']/501
salt_bridges['Occupancy'] = salt_bridges['count']/501                
print("done")
salt_bridges
        
        

done


,id,resid,restype,count,occupancy,Occupancy
0,243540,243,ARG,879,0,1.75449
1,243540,243,ARG,879,0,1.75449
2,243540,243,ARG,879,0,1.75449
3,243540,243,ARG,879,0,1.75449
4,434540,434,ARG,101,0,0.201597
5,265540,265,GLU,194,0,0.387226
6,237540,237,ARG,269,0,0.536926
7,440540,440,ARG,43,0,0.0858283


In [12]:
hydrogen_bonds['chain'] = 'A'
for i in range(len(hydrogen_bonds['count'])):
    if int(hydrogen_bonds.resid[i]) > 342:
        hydrogen_bonds.resid[i] = int(hydrogen_bonds.resid[i]) - 197
        hydrogen_bonds.chain[i] = "B"
salt_bridges['chain'] = 'A'
for i in range(len(salt_bridges['count'])):
    if int(salt_bridges.resid[i]) > 342:
        salt_bridges.resid[i] = int(salt_bridges.resid[i]) - 197
        salt_bridges.chain[i] = 'B'
hydrogen_bonds.to_csv("closed-rep2-hbonds.csv")
salt_bridges.to_csv("closed-rep2-sbridges.csv")
salt_bridges

,id,resid,restype,count,occupancy,Occupancy,chain
0,243540,243,ARG,879,0,1.75449,A
1,243540,243,ARG,879,0,1.75449,A
2,243540,243,ARG,879,0,1.75449,A
3,243540,243,ARG,879,0,1.75449,A
4,434540,237,ARG,101,0,0.201597,A
5,265540,265,GLU,194,0,0.387226,A
6,237540,237,ARG,269,0,0.536926,A
7,440540,243,ARG,43,0,0.0858283,A


In [13]:
salt_bridges

,id,resid,restype,count,occupancy,Occupancy,chain
0,243540,243,ARG,879,0,1.75449,A
1,243540,243,ARG,879,0,1.75449,A
2,243540,243,ARG,879,0,1.75449,A
3,243540,243,ARG,879,0,1.75449,A
4,434540,237,ARG,101,0,0.201597,A
5,265540,265,GLU,194,0,0.387226,A
6,237540,237,ARG,269,0,0.536926,A
7,440540,243,ARG,43,0,0.0858283,A


In [9]:
interactions = []
for i in range(501):
    data = open('reports/frame'+str(i)+'_report.xml', 'r').read() #abro el archivo xml
    data2 = xm.parse(data) #leo cuasi bien el fucking archivo
    bonds = dict(data2['report']['bindingsite']['interactions']) 
    for i in bonds.keys():
        if bonds.get(i) != None:
            if i not in interactions:
                interactions.append(i)
interactions
        

['hydrogen_bonds', 'salt_bridges', 'pi_cation_interactions', 'pi_stacks']

In [14]:
hydrogen_bonds
hydrogen_bonds.sort_values(by=['Occupancy'],ascending=False)


,id,resid,restype,donor_group,acceptor_group,count,Occupancy,chain
9,53036453,272,THR,O3,O2,500,0.998004,A
0,15986438,243,ARG,Ng+,N2,472,0.942116,A
11,48116483,243,ARG,Ng+,Npl,330,0.658683,A
2,64911618,244,VAL,Npl,O2,319,0.636727,A
1,15926489,243,ARG,Ng+,N2,287,0.572854,A
...,...,...,...,...,...,...,...,...
38,3266457,168,TYR,Nam,O3,1,0.00199601,A
40,16036489,244,VAL,Nam,N2,1,0.00199601,A
42,14866454,237,ARG,Ng+,O3,1,0.00199601,A
45,3946444,172,TYR,Nam,O3,1,0.00199601,A
